In [133]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import math
import os
BATCH_SIZE = 32

In [134]:
os.listdir("./training/training")

['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9']

In [135]:
train_dataset_path = './training/training'
test_dataset_path = './validation/validation'

In [136]:
# Result from mean_std.ipynb: 
# (mean, std) = (tensor([0.4363, 0.4328, 0.3291]), tensor([0.2129, 0.2075, 0.2038]))
mean = [0.4363, 0.4328, 0.3291]
std = [0.2129, 0.2075, 0.2038]
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)), 
    # transforms.CenterCrop((10, 20)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

In [137]:
train_dataset = torchvision.datasets.ImageFolder(root = train_dataset_path, transform = train_transforms)
test_dataset = torchvision.datasets.ImageFolder(root = test_dataset_path, transform = test_transforms)

In [147]:
train_loader = DataLoader(dataset = train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size=BATCH_SIZE, shuffle=True)

batch = next(iter(train_loader))
images, labels = batch
print(images.shape, labels.shape)

torch.Size([32, 3, 224, 224]) torch.Size([32])


In [ ]:
def example_transformed_images(dataset):
    batch = next(iter(DataLoader(dataset, batch_size=12, shuffle=True)))
    images, labels = batch
     
    grid = torchvision.utils.make_grid(images, nrow=3)
    plt.figure(figsize=(11, 11))
    plt.imshow(np.transpose(grid, (1, 2 , 0)))
    print('labels: ', labels)
# example_transformed_images(train_dataset)

In [140]:
def set_device():
    if torch.cuda.is_available():
        dev = "cuda:0"
    else:
        dev = "cpu"
    return torch.device(dev)

In [141]:
def train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs):
    device = set_device()
    
    for epoch in range(n_epochs):
        print("Epoch number %d " %(epoch+1))
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        
        for data in train_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            
            _, predicted = torch.max(outputs.data, 1)
            
            loss = criterion(outputs, labels)
              
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()
        
        epoch_loss = running_loss/len(train_loader)
        epoch_acc = 100.00 * running_correct / total
        
        # print(f'      -Training dataset. Got {running_correct} out of {total} images correctly ({epoch_acc}). Epoch loss: {epoch_loss}')
        print('      -Training dataset. Got %d out of %d images correctly (%.3f%%). Epoch loss: %.3f%%' 
                %(running_correct, total, epoch_acc, epoch_loss))
        
        evaluate_model(model, test_loader)
        
    print('Finish')
    
    return model

            

In [142]:
def evaluate_model(model, test_loader):
    model.eval()
    
    predicted_correctly_on_epoch = 0
    total = 0
    device = set_device()
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            
            outputs = model(images)
            
            _, predicted = torch.max(outputs.data, 1)
            
            predicted_correctly_on_epoch += (predicted == labels).sum().item()
    
    epoch_acc = 100.00 * predicted_correctly_on_epoch / total
    print('      -Testing dataset. Got %d out of %d images correctly (%.3f%%)' 
          %(predicted_correctly_on_epoch, total, epoch_acc))

    

In [143]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

resnet18_model = models.resnet18(pretrained=True)
num_features = resnet18_model.fc.in_features
number_of_classes = 10
resnet18_model.fc = nn.Linear(num_features, number_of_classes)
device = set_device()
resnet_18_model = resnet18_model.to(device)
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.SGD(resnet18_model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.003)

In [146]:
train_nn(resnet18_model, train_loader, test_loader, loss_fn, optimizer, 20)

Epoch number 1 
      -Training dataset. Got 1096 out of 1097 images correctly (99.909%). Epoch loss: 0.011%
      -Testing dataset. Got 241 out of 272 images correctly (88.603%)
Epoch number 2 
      -Training dataset. Got 1089 out of 1097 images correctly (99.271%). Epoch loss: 0.037%
      -Testing dataset. Got 227 out of 272 images correctly (83.456%)
Epoch number 3 
      -Training dataset. Got 1095 out of 1097 images correctly (99.818%). Epoch loss: 0.025%
      -Testing dataset. Got 233 out of 272 images correctly (85.662%)
Epoch number 4 
      -Training dataset. Got 1097 out of 1097 images correctly (100.000%). Epoch loss: 0.012%
      -Testing dataset. Got 237 out of 272 images correctly (87.132%)
Epoch number 5 
      -Training dataset. Got 1097 out of 1097 images correctly (100.000%). Epoch loss: 0.010%
      -Testing dataset. Got 239 out of 272 images correctly (87.868%)
Epoch number 6 
      -Training dataset. Got 1096 out of 1097 images correctly (99.909%). Epoch loss: 0

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  